In [9]:
import sys

import cv2 as cv
import numpy as np

from motion_detector.motion_detector import MotionDetector
from people_detector.people_detector import PeopleDetector
from utils.bbox_utils import crop_bboxes
from utils.view import view

## PROCESS VIDEO
*Here happens the magick*


In [10]:
def process_video(video_path: str, motion_detector: MotionDetector, people_detector: PeopleDetector, scale: float = 0.5, early_stop=None, starts_from=0):
    """
    Processes the input video.
    Parameters:
        video_path: Path to the video file.
        scale: Scaling factor for resizing frames.
        overlap_threshold: Threshold for merging overlapping detections using IoU.
        area_threshold: Minimum area for detected bounding boxes.
        people_detector: YOLO model to detect people.
        early_stop: stop after n frames
    """
    capture = cv.VideoCapture(video_path)
    iterations = 0
    while early_stop is None or iterations < early_stop:
        iterations += 1
        ret, frame = capture.read()
        if iterations <= starts_from:
            print("skipping frame {}".format(iterations))
            continue
        if not ret:
            break

        # frame = cv.resize(frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
        frame = cv.resize(frame, None, fx=scale, fy=scale, interpolation=cv.INTER_AREA)

        # detections, merged_detections, frame = mog2_movement_detection(frame, background_subtractor=back_sub, area_threshold=area_threshold,
        #                         overlap_threshold=overlap_threshold, draw=False)

        detections, merged_detections, frame = motion_detector(frame, draw=False)

        if not merged_detections:
            continue

        # drop area colum
        # cropped_frames = crop_bboxes(frame, bboxes=[md[:-1] for md in merged_detections])
        merged_detections = np.array(merged_detections)
        cropped_frames = crop_bboxes(frame, bboxes=merged_detections[:, :-1]) # a-la numpy

        # detect people on cropped frames

        # probs, bboxes, result = people_detector.detect(frame)
        # annotated_frame = result.plot()
        # view(annotated_frame)

        detections = people_detector.detect_on_frames(cropped_frames)
        for detection in detections:
            probs, _, result = detection
            print(probs)
            if len(probs) == 0:
                continue
            annotated_frame = result.plot()
            # Image.fromarray(annotated_frame).save('./runs/detect/{}.png'.format(iterations))

            view(annotated_frame)

        # for frame in cropped_frames:
        #     stop = view(frame, scale=scale)
        #     if not stop:
        #         break
    else:
        print(f"Stopped after {iterations} frames due early stopping condition.")

    capture.release()
    cv.destroyAllWindows()

    if sys.platform == 'darwin':
        for _ in range(4):
            cv.waitKey(1)

    cv.destroyAllWindows()



## Main

In [11]:
yolosize = 'x'
yolo11 = PeopleDetector(f"yolo11{yolosize}.pt", verbose=False,)
yolo11.to('cpu')


overlap_threshold=0.0005
area_threshold=700
motion_detector = MotionDetector(area_threshold=area_threshold, overlap_threshold=overlap_threshold)


# probs, bboxes, _ = yolo11.detect("The-Office-HD-Background.jpg")
# print('confidence scores', probs)
# print('bboxes', bboxes)

In [ ]:
video_path = 'SamsungGear360.mp4'
process_video(video_path, motion_detector, yolo11, scale=0.5) # , starts_from=3000, early_stop=3500)

# People Detection TEST


Define the base functions for people detection. <br/>
It Takes as input an array of frames containing the motion detected by MOG2 above

In [ ]:
# imports

In [ ]:
# Load a model
model = PeopleDetector("./yolo11x.pt")  # load an official model, or use local path

In [ ]:
def test_yolo11(video_path: str, scale: float = 0.5):
    # Open the video file
    cap = cv.VideoCapture(video_path)

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLO inference on the frame
            # frame = cv.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv.INTER_LINEAR)
            results = model(frame)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            annotated_frame = cv.resize(annotated_frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
            # Display the annotated frame
            cv.imshow("YOLO Inference", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    cv.destroyAllWindows()

## Main test YOLO11

In [ ]:
test_yolo11("SamsungGear360.mp4", scale=0.25)